# HIGH-DIMENSIONAL METRICS IN R

## 2. How to get started

In [56]:
import Pkg; Pkg.add("HDMjl")

or

In [38]:
import Pkg; Pkg.add(url = "https://github.com/d2cml-ai/HDMjl.jl")

    Updating git-repo `https://github.com/d2cml-ai/HDMjl.jl`


   Resolving package versions...


  No Changes to `C:\Users\User\.julia\environments\v1.8\Project.toml`


  No Changes to `C:\Users\User\.julia\environments\v1.8\Manifest.toml`


In [17]:
run(`cmd /c start  $(Base.julia_cmd()) --banner=no -i myscript.jl`)

Process(`cmd /c start 'C:\Users\User\AppData\Local\Programs\julia-1.8.1\bin\julia.exe' -Cnative '-JC:\Users\User\AppData\Local\Programs\julia-1.8.1\lib\julia\sys.dll' -g1 --color=yes --banner=no -i myscript.jl`, ProcessExited(0))

In [299]:
using HDMjl, CodecXz, RData, DataFrames, StatsModels, Statistics, Distributions, PrettyTables

In [22]:
using UUIDs

In [21]:
Pkg.add("UUIDs")

    Updating registry at `C:\Users\User\.julia\registries\General`


    Updating git-repo `https://github.com/JuliaRegistries/General.git`


   Resolving 

package versions...
    Updating `C:\Users\User\.julia\environments\v1.8\Project.toml`


  [cf7118a7] + UUIDs
  No Changes to `C:\Users\User\.julia\environments\v1.8\Manifest.toml`


In [23]:
uuid4()

UUID("70f5ce91-05bc-4285-b635-19f216e408f8")

In [2]:
import Pkg

In [6]:
Pkg.rm("HDMjl")

    Updating `C:\Users\User\.julia\environments\v1.8\Project.toml`


  [8de29b41] - HDMjl v0.0.10 `E:/causal_ml/hdm_paper/HDMjl.jl`
    Updating `C:\Users\User\.julia\environments\v1.8\Manifest.toml`


  [8de29b41] - HDMjl v0.0.10 `E:/causal_ml/hdm_paper/HDMjl.jl`


In [7]:
Pkg.develop(path = "E:/causal_ml/hdm_paper/HDMjl.jl")

   Resolving package versions...


    Updating `C:\Users\User\.julia\environments\v1.8\Project.toml`


  [8de29b41] + HDMjl v0.0.10 `E:/causal_ml/hdm_paper/HDMjl.jl`
    Updating `C:\Users\User\.julia\environments\v1.8\Manifest.toml`


  [8de29b41] + HDMjl v0.0.10 `E:/causal_ml/hdm_paper/HDMjl.jl`


In [9]:
#Pkg.update("HDMjl")

In [43]:
using HDMjl

## 3. Prediction using Approximate Sparsity

### 3.2. A Joint Significance Test for Lasso Regression.

In [10]:
using Random
Random.seed!(1234);
n = 100;
p = 100;
s = 3;
X = randn(n, p);
beta = vcat(fill(5, s), zeros(p - s));
Y = X * beta + randn(n);

In [11]:
a = rlasso(X, Y, post = false)
length(a["residuals"])

100

In [12]:
post_lasso_reg = rlasso(X, Y, post = true) #now use post-lasso
post_lasso_reg["coefficients"]'

1×101 adjoint(::Vector{Float64}) with eltype Float64:
 -0.00682754  5.00958  4.93178  5.17705  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [11]:
level = 0.95
a = (1 - level)/2
a = [a, 1 - a]
quantile.(Normal(), a)
round.(a; digits = 3)

UndefVarError: UndefVarError: Normal not defined

## 4. Inference on Target Regression Coefficients

### 4.1. Intuition for the Orthogonality Principle in Linear Models via Partialling Out.

### 4.2. Inference: Confidence Intervals and Significance Testing. The function rlassoEffects

In [576]:
using Random
Random.seed!(1234);
n = 100;
p = 100;
s = 3;
x = randn(n, p);
beta = vcat(fill(3, s), zeros(p - s));
y =1 .+ X * beta + randn(n);

In [577]:
rlassoEffects(x, y)

MethodError: MethodError: no method matching isless(::Int64, ::Set{Int64})
Closest candidates are:
  isless(::Union{AbstractChar, AbstractString, Number}, !Matched::CategoricalArrays.CategoricalValue) at C:\Users\User\.julia\packages\CategoricalArrays\tJ8hD\src\value.jl:157
  isless(::Real, !Matched::AbstractFloat) at operators.jl:185
  isless(::Real, !Matched::Union{StatsBase.PValue, StatsBase.TestStat}) at C:\Users\User\.julia\packages\StatsBase\XgjIN\src\statmodels.jl:91
  ...

In [601]:
any(t)

false

In [602]:
#Set(index)
Set([1:10;])

Set{Int64} with 10 elements:
  5
  4
  6
  7
  2
  10
  9
  8
  3
  1

In [616]:
index = 1:size(x, 2) 
I3 = nothing
x = Matrix(x)
y = Matrix(y[:, :])

# if Set(index) > 2
#     k = p = length(index)
#     # all(all())
# else
#     k = p = sum(index)
# end

k = p1 = length(index)
n, p0 = size(x)

coefficients = zeros(k)
se = zeros(k)
t = zeros(k)
lasso_reg = Dict()

reside = zeros(n, p1)
residv = zeros(n, p1)

selection_matrix = zeros(k
, size(x)[2])
# coef_mat = Dict()
# for i in 1:k
#     d = x[:, index[i]]
#     xt = x[:, Not(index[i])]
#     if isnothing(I3)
#         I3m = I3
#     else
#         I3m = I3[Not(index[i])]
#         lasso_reg[i] = try
#             rlassoEffect(xt, y, d, method = method, I3 = I3m, post = post)
#         catch
#             "try-error"
#         end
#     end
#     if lasso_regs[i] == "try-error"
#         continue
#     else
#         coefficients[i] = lasso_regs[i]["alpha"]
#         se[i] = lasso_regs[i]["se"]
#         t[i] = lasso_regs[i]["t"]
#         coef_mat[i] = lasso_reg["coefficients_reg"]
#         reside[:, i] = lasso_regs[i]["residuals"]["epsilon"]
#         residv[:, i] = lasso_regs[i]["residuals"]["v"]
#         selection_matrix[Not(index[i]), i] = lasso_reg["selection_index"]
#     end
# end

100×100 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮              ⋱            ⋮                   
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

### 4.3. Application: the effect of gender on wage

### 4.4. Application: Estimation of the treatment effect in a linear model with many confounding factors

In [545]:
function summary_effect(object)
    if length(object) != 0
        k = length(object["coefficients"])
        table = zeros(k, 4)
        table[:, 1] .= object["coefficients"]
        table[:, 2] .= object["se"]
        table[:, 3] .= table[:, 1]/table[:, 2]
        table[:, 4] .= 2 * cdf(Normal(), -abs.(table[:, 3]))
        print(""","Estimates and significance testing of the effect of target variables",""", 
                "\n")
        pretty_table(table, show_row_number = true, header = ["Estimate.", "Std. Error", "t value", "Pr(>|t|)"], tf = tf_borderless)
        print("---", "\n", "Signif. codes:","\n", "0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1")
        print("\n")
    else
        print("No coefficients\n")
    end
end

summary_effect (generic function with 1 method)

In [546]:
using CodecXz
using RData
using DataFrames
url = "https://github.com/cran/hdm/raw/master/data/GrowthData.rda";
GrowthData = load(download(url))["GrowthData"];
y = GrowthData[:, 1];
d = GrowthData[:, 3];
X = Matrix(GrowthData[:, Not(1, 2, 3)]);

In [547]:
lasso_effect = rlassoEffect(X, y, d, method = "partialling out");

In [548]:
summary_effect(lasso_effect)

,"Estimates and significance testing of the effect of target variables",
  Row   Estimate.   Std. Error   t value      Pr(>|t|) 

    1    -0.05333    0.0143283    -3.722   0.000197655
---
Signif. codes:
0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [549]:
doublesel_effect = rlassoEffect(X, y, d, method = "double selection");

In [550]:
summary_effect(doublesel_effect)

,"Estimates and significance testing of the effect of target variables",
  Row    Estimate.   Std. Error    t value    Pr(>|t|) 

    1   -0.0453558     0.018656   -2.43116   0.0150506
---
Signif. codes:
0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


## 5. Instrumental Variable Estimation in a High-Dimensional Setting

### 5.2. Application: Economic Development and Institutions.

In [542]:
function confintIV(object, level = 0.95)
    n = object["sample_size"]
    k = length(object["coefficients"])
    cf = object["coefficients"]
    #pnames <- names(cf)
    # if (missing(parm)) 
    #     parm <- pnames else if (is.numeric(parm)) 
    #       parm <- pnames[parm]
    a = (1 - level)/2
    a = [a, 1 - a]
    fac = quantile.(Normal(), a)
    pct = string.(round.(a; digits = 3)*100, "%")
    ses = object["se"]
    c_i = []
    for i in 1:length(cf)
        if i == 1
            c_i = (cf[i] .+ ses * fac)[:,:]'
        else|
            c_i = vcat(c_i, (cf[i] .+ ses * fac)[:,:]')
        end
    end
    #ci = NamedArray(c_i, (1:size(c_i)[1], pct))
    ci = pretty_table(c_i; header = pct, show_row_number = true, tf = tf_borderless)
    #return c_i;;
end

confintIV (generic function with 2 methods)

In [294]:
# using PrettyTables
# object = AJR_Xselect
# level = 0.95

# n = object["sample_size"]
# k = length(object["coefficients"])
# cf = object["coefficients"]
# #pnames <- names(cf)
# # if (missing(parm)) 
# #     parm <- pnames else if (is.numeric(parm)) 
# #       parm <- pnames[parm]
# a = (1 - level)/2
# a = [a, 1 - a]
# fac = quantile.(Normal(), a)
# pct = string.(round.(a; digits = 3)*100, "%")
# ses = object["se"]
# c_i = []
# for i in 1:length(cf)
#     if i == 1
#         c_i = (cf[i] .+ ses * fac)[:,:]'
#     else|
#         c_i = vcat(c_i, (cf[i] .+ ses * fac)[:,:]')
#     end
# end
# #ci = NamedArray(c_i, (1:size(c_i)[1], pct))
# ci = pretty_table(c_i; header = pct, show_row_number = true, tf = tf_borderless)
# ci

In [541]:
function summaryIV(object)
    if length(object) != 0
        k = length(object["coefficients"])
        table = zeros(k, 4)
        table[:, 1] .= object["coefficients"]
        table[:, 2] .= object["se"]
        table[:, 3] .= table[:, 1]/table[:, 2]
        table[:, 4] .= 2 * cdf(Normal(), -abs.(table[:, 3]))
        print("Estimates and Significance Testing of the effect of target variables in the IV regression model", 
                "\n")
        pretty_table(table, show_row_number = true, header = ["coeff.", "se.", "t-value", "p-value"], tf = tf_borderless)
        print("---", "\n", "Signif. codes:","\n", "0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1")
        print("\n")
    else
        print("No coefficients\n")
        #table = []
    end
    #return table;;
end

summaryIV (generic function with 1 method)

In [464]:
using Statistics, StatsModels
url = "https://github.com/cran/hdm/raw/master/data/AJR.rda";
AJR = load(download(url))["AJR"];
y = AJR[!,"GDP"]
d = AJR[!,"Exprop"]
z = AJR[!,"logMort"];
x_formula = @formula(GDP ~ -1 + Latitude + Latitude2 + Africa + Asia + Namer + Samer
    + Latitude*Latitude2 + Latitude*Africa + Latitude*Asia + Latitude*Namer + Latitude*Samer
    + Latitude2*Africa + Latitude2*Asia + Latitude2*Namer + Latitude2*Samer
    + Africa*Asia + Africa*Namer + Africa*Samer
    + Asia*Namer + Asia*Samer
    + Namer*Samer)
x_dframe = ModelFrame( x_formula, AJR)
x1 = ModelMatrix(x_dframe)
x = x1.m
size(x)

(64, 21)

In [465]:
AJR_Xselect  = rlassoIV(x, d, y, z, select_X=true, select_Z=false);

In [544]:
summaryIV(AJR_Xselect)

Estimates and Significance Testing of the effect of target variables in the IV regression model
  Row    coeff.       se.   t-value    p-value 

    1   1.27623   0.87538   1.45791   0.144865
---
Signif. codes:
0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


In [543]:
confintIV(AJR_Xselect)

  Row        2.5%     97.5% 

    1   -0.439485   2.99194


### 5.3. Application: Impact of Eminent Domain Decisions on Economic Outcomes.

In [56]:
using Statistics
url = "https://github.com/cran/hdm/raw/master/data/EminentDomain.rda";
EminentDomain = load(download(url))["EminentDomain"];
z = EminentDomain["logGDP"]["z"];
x = EminentDomain["logGDP"]["x"];
d = EminentDomain["logGDP"]["d"];
y = EminentDomain["logGDP"]["y"];
x = x[:, (mean(x, dims = 1) .> 0.05)'];
z = z[:, (mean(z, dims = 1) .> 0.05)'];
rlassoIV(x, d, y, z, select_X = false, select_Z = true)

Dict{String, Any} with 4 entries:
  "samplesize"   => 312
  "se"           => [2.32377]
  "residuals"    => [0.114175; 0.056835; … ; 0.256009; 0.305468;;]
  "coefficients" => [-0.451027;;]

In [534]:
a = rlassoIV(x, d, y, z, select_X = true, select_Z = true)

Dict{String, Any} with 5 entries:
  "se"           => 0.125636
  "sample_size"  => 64
  "vcov"         => 0.0157844
  "residuals"    => [0.407001, 0.734715, 0.325353, -0.0492531, 0.0376143, -0.40…
  "coefficients" => 0.337263

## 6. Inference on Treatment Effects in a High-Dimensional Setting

### 6.3. Application: 401(k) plan participation.

In [18]:
url = "https://github.com/cran/hdm/raw/master/data/pension.rda";
pension = load(download(url))["pension"];
y = pension[:, "tw"];
d = pension[:, "p401"];
z = pension[:, "e401"];
X = Matrix(pension[:, ["i2", "i3", "i4", "i5", "i6", "i7", "a2", "a3", "a4", "a5", "fsize", "hs", "smcol", "col", "marr", "twoearn", "db", "pira", "hown"]]);
rlassoATE(X, d, y)

Dict{String, Any} with 5 entries:
  "se"          => 1930.68
  "individual"  => [-30618.3, -57537.6, -71442.9, 21383.3, -2.32925e5, 3.40765e…
  "sample_size" => 9915
  "te"          => 10180.1
  "type"        => "ATE"

In [51]:
rlassoATET(X, d, y)

Dict{String, Any} with 5 entries:
  "se"          => 2944.43
  "individual"  => [-21536.4, -52877.2, -1.44867e5, -2739.29, -307741.0, 7.3912…
  "sample_size" => 9915
  "te"          => 12628.5
  "type"        => "ATET"

In [52]:
rlassoLATE(X, d, y, z)

Dict{String, Any} with 5 entries:
  "se"          => 2326.9
  "individual"  => [-50526.8, -1.39158e5, -1.37102e5, 38508.0, -6.5644e5, 7.943…
  "sample_size" => 9915
  "te"          => 12992.1
  "type"        => "LATE"

In [53]:
rlassoLATET(X, d, y, z)

Dict{String, Any} with 5 entries:
  "se"          => 3645.28
  "individual"  => [-35580.5, -90558.0, -1.83628e5, -5303.13, -8.0766e5, 1.8866…
  "sample_size" => 9915
  "te"          => 15323.2
  "type"        => "LATET"